# Conversational Task Agent

Taskbot that assist users in multi-step tasks, such as baking a 
birthday  cake  or  fixing  a  scratch  on  a  car  —  and  adapt  those  instructions  based  on  the 
resources  and  tools  available  to  the  customer.  If,  for  example,  a  customer  ran  out  of  an 
ingredient  halfway  through  a  recipe  or  didn’t  have  a  specific  tool  for  a  DIY  project,  the 
taskbot had to adjust the plan and suggest possible solutions.

## Phase 1 - (Task Retriever): Implementing a search index for recipes with OpenSearch

In [2]:
from modules.Indexing import Indexing

indexer = Indexing()

indexer.calculateAndStoreEmbeddings()
print("Embeddings stored and encoded")
indexer.createOpenSearchMappings()
print("Mappings created")
indexer.readAndStoreRecipesFromFile("./receitas.json")
print("Recipes stored")

Embeddings stored and encoded
Index already existed. Nothing to be done.
Mappings created
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
updated
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
created
create

In [5]:
from modules.Search import Search

search = Search()

# search.SearchTitleTxt("chicken")
# search.SearchDescriptionTxt("Chicken")
# search.SearchTitleEmbeddings("Chicken")
# search.SearchDescriptionEmbeddings("Chicken")
# search.SearchRecipeTime(30, 5)
# search.SearchRecipeIngredients("chicken, rice")
# search.SearchRecipeDifficulty("easy")
# search.SearchRecipeNutrition("protein")

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [5]:
from modules.SearchBuilder import SearchBuilder
import json
import pprint as pp

In [6]:
# search for vegetarian pesto pasta
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setResultLength(5)
searchBuilder.setMandatoryTags(['Vegan', 'Vegetarian'])
searchBuilder.setOptionalTags(['Gluten Free'])

# print debug
#pp.pprint(    searchBuilder.queryBuilder)

response = searchBuilder.Search("Pesto pasta")
pp.pprint(response) # print

{'size': 5, '_source': ['doc_id', 'contents'], 'query': {'bool': {'must': [{'terms': {'tags': ['Vegan', 'Vegetarian']}}], 'should': [{'terms': {'tags': ['Gluten Free']}}, {'multi_match': {'query': 'Pesto pasta', 'fields': ['title', 'description']}}]}}}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '517',
                    '_index': 'user212',
                    '_score': 5.796566,
                    '_source': {'contents': "{'displayName': 'Pesto Sauce from "
                                            "Scratch', 'description': None, "
                                            "'canonicalName': 'Pesto Sauce "
                                            "from Scratch', 'prepTimeMinutes': "
                                            "None, 'cookTimeMinutes': None, "
                                            "'totalTimeMinutes': 10, "
                                            "'cookingMethod': None, "
                          

In [7]:
# search for Yaki udon
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("mushrooms")
searchBuilder.excludeIngredients("brocolli")
searchBuilder.setOptionalTags(["Vegetarian"])
searchBuilder.setTimeFrame(20, 15)

#pp.pprint(    searchBuilder.queryBuilder)

response = searchBuilder.Search("Yaki udon with at home ingredients")
pp.pprint(response)# print

{'size': 5, '_source': ['doc_id', 'contents'], 'query': {'bool': {'must': [{'match': {'ingredients': {'query': 'mushrooms', 'operator': 'or'}}}, {'range': {'time': {'gte': 5, 'lte': 35}}}], 'must_not': [{'match': {'ingredients': {'query': 'brocolli', 'operator': 'or'}}}], 'should': [{'terms': {'tags': ['Vegetarian']}}, {'multi_match': {'query': 'Yaki udon with at home ingredients', 'fields': ['title', 'description']}}]}}}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [{'_id': '569',
                    '_index': 'user212',
                    '_score': 6.09216,
                    '_source': {'contents': "{'displayName': 'New York Strip "
                                            'Steak with Red Wine Mushroom '
                                            "Sauce', 'description': None, "
                                            "'canonicalName': 'New York Strip "
                                            'Steak with Red Wine Mushroom '
   

In [8]:
# search for Dumpling Miso Soup Ramen
searchBuilder = SearchBuilder()
searchBuilder.setSourceAsIdAndContent()
searchBuilder.setIngredients("dumplings")
searchBuilder.setMandatoryTags(["Vegan", "Vegetarian", "Gluten Free"])
searchBuilder.setOptionalTags(["Main Dishes", "Japanese"])

#pp.pprint(    searchBuilder.queryBuilder)

response = searchBuilder.Search("Dumpling Miso Soup Ramen")
pp.pprint(response)# print

{'size': 5, '_source': ['doc_id', 'contents'], 'query': {'bool': {'must': [{'match': {'ingredients': {'query': 'dumplings', 'operator': 'or'}}}, {'terms': {'tags': ['Vegan', 'Vegetarian', 'Gluten Free']}}], 'should': [{'terms': {'tags': ['Main Dishes', 'Japanese']}}, {'multi_match': {'query': 'Dumpling Miso Soup Ramen', 'fields': ['title', 'description']}}]}}}
{'_shards': {'failed': 0, 'skipped': 0, 'successful': 4, 'total': 4},
 'hits': {'hits': [],
          'max_score': None,
          'total': {'relation': 'eq', 'value': 0}},
 'timed_out': False,
 'took': 3}


In [10]:
from opensearchpy import OpenSearch

host = 'api.novasearch.org'
port = 443
user = 'user212'
password = 'soO-2518'
index_name = user
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = (user, password),
    url_prefix = 'opensearch',
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

if client.indices.exists(index=index_name):
    # Delete the index.
    response = client.indices.delete(
        index = index_name,
        timeout = 600
    )
    print('\nDeleting index:')
    print(response)


Deleting index:
{'acknowledged': True}
